# Run application

This page provides a detailed explanation of the options available for running a FastAPI application.

In [7]:
import requests

## Image

Here we'll focus on the image that we'll primarily use for experimenting with FastAPI.  This is the default image used in most FastAPI examples.

### Dockerfile

In the next cell is the docker file I am using for this example.

In [ ]:
%%writefile run_application_files/dockerfile
FROM python:3.11
COPY requrements.txt requrements.txt
RUN pip3 install -r requrements.txt
EXPOSE 8000

Overwriting fastapi/run_application_files/dockerfile


### `requrements.txt`

Python libraries you only needed to run the `fastapi` server. It is supposed to be used in the [dockerfile](#sec-dockerfile) described above.

In [ ]:
%%writefile fastapi/run_application_files/requrements.txt
fastapi==0.103.1
uvicorn==0.23.2

Overwriting fastapi/run_application_files/requrements.txt


### Build the image

Image with name `fastapi_experiment`, created in the following cell, will be used in the other subsections of the fastapi section.

In [ ]:
!docker build -t fastapi_experiment\
    ./fastapi/run_application_files/ &> /dev/null

## Runtime update

The most convenient approach to experimenting with a container that contains fastapi is to dynamically swap the program, enabling the execution of multiple examples within a single container. For this purpose, I typically establish a connection between the utilized execution Python file and the container as a volume. This ensures that any changes made on the computer are promptly reflected within the container.

To implement this functionality, it is necessary to run uvicorn with the `--reload` flag. This flag enables uvicorn to monitor changes in the program and update accordingly.

---

The following cells start an application that returns `initial line`. 

In [15]:
%%writefile ./run_application_files/reload.py
from fastapi import FastAPI

my_first_app = FastAPI()

@my_first_app.get("/")
def say_hello():
    return "initial line"

Overwriting ./run_application_files/reload.py


Note that the `uvicorn` call here includes the `--reload` option.

In [16]:
!docker run --rm -itd\
    --name test_container\
    -v ./run_application_files/reload.py:/reload.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 --reload reload:my_first_app

f70c9566a8c067d1d20008d9204876e5696a48823218b5a6ac02aa12458a59a8


As expected, a request to this API returns `initial line`. 

In [17]:
requests.get("http://localhost:8000/").content

b'"initial line"'

Now, without restarting the container, simply change the file containing your application - it will now return `updated line`. 

In [18]:
%%writefile ./run_application_files/reload.py
from fastapi import FastAPI

my_first_app = FastAPI()

@my_first_app.get("/")
def say_hello():
    return "updated line"

Overwriting ./run_application_files/reload.py


By checking api result we'll got `updated line`.

In [19]:
requests.get("http://localhost:8000/").content

b'"updated line"'

Therefore, the API was refreshed without restarting the container. 

In [20]:
!docker stop test_container

test_container


## Run as `__main__`

Sometimes it's useful to be able to start a program by simply running it as a Python script, i.e. without bothering to call `uvicorn`.

All you have to do is run `uvicorn.run(<fast_api object>, ...)`.

---

The next cell shows a program using such an approach.

In [21]:
%%writefile ./run_application_files/reload.py
import uvicorn
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def say_hello():
    return "I'm started from __main__"

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

Overwriting ./run_application_files/reload.py


To run it now we just use the command `python3 <script_name>.py`.

In [23]:
!docker run --rm -itd\
    --name test_container\
    -v ./run_application_files/reload.py:/reload.py\
    -p 8000:8000 \
    fastapi_experiment \
    python3 reload.py

054cc3956c27ad9c72c3ec7e960ccd92eed4b8ac9544c751af6040a6dab3b2db


Let's ask api to make sure everything is working.

In [24]:
!curl localhost:8000

"I'm started from __main__"

In [25]:
!docker stop test_container

test_container
